In [7]:
import numpy as np
import matplotlib.pyplot as plt
import visualization
import os
from gta_math import points_to_homo, ndc_to_view, construct_proj_matrix, view_to_world, construct_view_matrix
from visualization import load_depth
import progressbar
from pointcloud_to_voxelmap import pointclouds_to_voxelmap
from joblib import Parallel, delayed
from configparser import ConfigParser
from PIL import Image
import pickle
from voxelmaps import camera_to_pointcloud, load_scene_db_data, get_main_image_name, scene_to_pointcloud, scene_to_voxelmap, NoMainImageException, scene_to_voxelmap_with_map, get_main_image
import voxelmaps
import time

In [2]:
ini_file = "gta-postprocessing.ini"
visualization.multi_page = False
visualization.ini_file = ini_file

conn = visualization.get_connection()
cur = conn.cursor()

CONFIG = ConfigParser()
CONFIG.read(ini_file)
in_directory = CONFIG["Images"]["Tiff"]
out_directory = CONFIG["Images"]["MlDatasetVoxel"]
out_root_directory = r'D:\Projects\GTAVisionExport_postprocessing'


In [3]:
run_id = 6

cur.execute("""SELECT DISTINCT scene_id \
  FROM snapshots \
  WHERE run_id = {} \
  """.format(run_id))

scenes = []
for row in cur:
    res = dict(row)
    scenes.append(res)

image_names = {}

print('There are {} scenes'.format(len(scenes)))


There are 8438 scenes


### functions for each scene

In [15]:
# import time
voxelmaps.MAX_DISTANCE = 30 # in meters, meaning I care only for point up to 20 meters

points_cache = {}

def get_base_name(name):
    return os.path.basename(os.path.splitext(name)[0])


def convert_tiff(in_directory, out_directory, out_name, name):
    # throws OSError
    out_format = 'jpg'
    outfile = os.path.join(out_directory, "{}.{}".format(out_name, out_format))
    if os.path.exists(outfile):
        return outfile
    infile = os.path.join(in_directory, name)
    im = Image.open(infile)
    im = im.convert(mode="RGB")
    # print("Generating new format for {} to new file {}".format(name, out_name))
    im.save(outfile)
    return outfile

def generate_frustum_points(proj_matrix):
    if proj_matrix.tobytes() in points_cache:
        return points_cache[proj_matrix.tobytes()].copy()

    x_min = -1  # NDC corners
    x_max = 1  # NDC corners
    y_min = -1  # NDC corners
    y_max = 1  # NDC corners
    x_range = 240  # X output size of NN
    y_range = 160  # Y output size of NN
    z_range = 100  # Z output size of NN
    z_meters_min = 1.5
    z_meters_max = voxelmaps.MAX_DISTANCE
    # z min calc
    z_min = proj_matrix @ [1, 1, -z_meters_max, 1]
    z_min = z_min[2] / z_min[3]
    # z max calc
    z_max = proj_matrix @ [1, 1, -z_meters_min, 1]
    z_max = z_max[2] / z_max[3]
    x_bin = (x_max - x_min) / x_range
    y_bin = (y_max - y_min) / y_range
    z_bin = (z_max - z_min) / z_range
    X, Y, Z, W = np.mgrid[x_min:x_max:x_bin, y_min:y_max:y_bin, z_min:z_max:z_bin, 1:2:1]  # W is here as homo coordinate
    positions = np.vstack([X.ravel(), Y.ravel(), Z.ravel(), W.ravel()])
    
    points_cache[proj_matrix.tobytes()] = positions
    return positions

def convert_scene_to_ndc_voxelmap(cameras):
    # here I generate points cuboid which will be output of NN.
    # these points will be transferred into camera view frustum
    # for this frustum, I take voxel values of calculated voxelmap
    # I use these voxel values as values for cuboid in NDC, the NN output
    voxels, values, map_voxel_size, map_obj = scene_to_voxelmap_with_map(cameras)
    cam = get_main_image(cameras)
    proj_matrix = cam['proj_matrix']
    view_matrix = cam['view_matrix']
    start = time.time()
    positions = generate_frustum_points(proj_matrix)
    end = time.time()
    print('time diff:', end - start)
    points_view = ndc_to_view(positions, proj_matrix)
    points_world = view_to_world(points_view, view_matrix)
    # here I find corresponding voxels for generated points, by obtaining voxelmap reference
    voxel_values = map_obj.get_voxels(points_world[0:3, :], np.zeros((points_world.shape[1], 1)))
    occupied_ndc_positions = positions[:, voxel_values > 0]
    return occupied_ndc_positions

    
def convert_scene_to_img_and_voxelmap(in_directory, out_directory, scene_id):
    if 'pbar' in globals() and 'counter' in globals():
        global counter
        counter += 1
        pbar.update(counter)

    # start = time.time()
    cameras = load_scene_db_data(scene_id)
    try:
        image_name = get_main_image_name(cameras)
        rgb_outfile = convert_tiff(in_directory, out_directory, image_name, image_name+'.tiff')
        outfile = os.path.join(out_directory, "{}.bin".format(image_name))

        if os.path.exists(outfile):
            return
        
        occupied_ndc_positions = convert_scene_to_ndc_voxelmap(cameras)
        occupied_ndc_positions.tofile(outfile)
        
        image_names[scene_id] = (rgb_outfile, outfile)
    except (OSError, NoMainImageException) as e:
        print('No main image for scene {}, skipping.'.format(scene_id))


### actually running the extraction

In [ ]:
workers = 10

widgets = [progressbar.Percentage(), ' ', progressbar.Counter(), ' ', progressbar.Bar(), ' ',
           progressbar.FileTransferSpeed()]

pbar = progressbar.ProgressBar(widgets=widgets, maxval=len(scenes)).start()
counter = 0

Parallel(n_jobs=workers, backend='threading')(delayed(convert_scene_to_img_and_voxelmap)(in_directory, out_directory, i['scene_id']) for i in scenes)


  0% 13 |                                                           | 121.5 B/s

time diff: 0.2249898910522461


### generating filenames list

In [ ]:
filenames = []
for scene_id, (rgb_outfile, outfile) in image_names.items():
    rgb_name = os.path.relpath(os.path.join(out_directory, rgb_outfile), start=out_root_directory)
    rgb_name = rgb_name.replace('\\', '/')
    voxelmap_name = os.path.relpath(os.path.join(out_directory, outfile), start=out_root_directory)
    voxelmap_name = depth_name.replace('\\', '/')
    filenames.append([rgb_name, depth_name])

    # r je abych nemusel psát zpětná lomítka
with open(os.path.join(out_root_directory, 'whole-voxel-gta.csv'), mode='w+', newline='') as f:
    csv.writer(f).writerows(filenames)

random.shuffle(filenames)

train_ratio = 0.8
train_threshold = int(train_ratio * len(image_names))
train_filenames = filenames[:train_threshold]
test_filenames = filenames[train_threshold:]

with open(os.path.join(out_root_directory, 'train-voxel-gta.csv'), mode='w+', newline='') as f:
    csv.writer(f).writerows(train_filenames)
with open(os.path.join(out_root_directory, 'test-voxel-gta.csv'), mode='w+', newline='') as f:
    csv.writer(f).writerows(test_filenames)


### running extraction for one scene

In [16]:
convert_scene_to_img_and_voxelmap(in_directory, out_directory, scenes[0]['scene_id'])

  0% 26 |                                                           |  30.6 s/B

time diff: 0.19000768661499023


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:68: RuntimeWarning: invalid value encountered in greater


In [17]:
# just some timing experiments
for i in range(5):
    convert_scene_to_img_and_voxelmap(in_directory, out_directory, scenes[i]['scene_id'])

  0% 27 |                                                           |  30.1 s/B

time diff: 0.03800010681152344


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:68: RuntimeWarning: invalid value encountered in greater
  0% 28 |                                                           |  29.4 s/B

time diff: 0.03700137138366699


  0% 29 |                                                           |  28.8 s/B

time diff: 0.0390017032623291


  0% 30 |                                                           |  28.2 s/B

time diff: 0.037989139556884766


  0% 31 |                                                           |  27.6 s/B

time diff: 0.038001060485839844


In [12]:
prev = None
for i in range(5):
    cameras = load_scene_db_data(scenes[i]['scene_id'])
    cam = get_main_image(cameras)
    proj_matrix = cam['proj_matrix']
    if prev is not None:
        print(np.array_equal(prev, proj_matrix))
    prev = proj_matrix

True
True
True
True


In [10]:
prev

NameError: name 'prev' is not defined